In [1]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\ADMIN\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)


In [3]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
#print(df.content.unique())
df.head()

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 1

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [7]:
type(df)

pandas.core.frame.DataFrame

In [6]:
# Convert to list
data = df.content.values.tolist()
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


C:\Users\ADMIN\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [9]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# Faster way to get a sentence clubbed as 


In [25]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [29]:
id2word[0]

'addition'

In [30]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('whatev', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [ ]:
# Build LDA model
n=2
for n in range(1,20,2):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=n, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



In [32]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"not" + 0.009*"would" + 0.008*"do" + 0.008*"line" + '
  '0.007*"organization" + 0.007*"write" + 0.006*"be" + 0.006*"article" + '
  '0.006*"know" + 0.005*"say"'),
 (1,
  '0.210*"ax" + 0.015*"max" + 0.007*"window" + 0.006*"line" + 0.006*"card" + '
  '0.005*"file" + 0.005*"use" + 0.005*"organization" + 0.004*"thank" + '
  '0.004*"drive"')]


In [33]:

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.519875232711755

Coherence Score:  0.5530051029070329


In [34]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\ADMIN\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.219813  0.0       1        1  81.035820
1     -0.219813  0.0       2        1  18.964186, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
4146  Default  64252.000000            ax  64252.000000  30.0000  30.0000
3721  Default   4659.000000           max   4659.000000  29.0000  29.0000
215   Default   2416.000000        window   2416.000000  28.0000  28.0000
82    Default  16458.000000           not  16458.000000  27.0000  27.0000
60    Default   2097.000000          card   2097.000000  26.0000  26.0000
738   Default   1859.000000          file   1859.000000  25.0000  25.0000
317   Default   4474.000000           use   4474.000000  24.0000  24.0000
43    Default   2435.000000         thank   2435.000000  23.0000  23.0000
263   Default   2091.000000         drive   2091.000000  22.0000  22.0000
206   Default   3252.000000        system   3252.000000  21.0000  21.0000
21    Default  12336.000000          line  12336.000000  20.0000  20.0000
141   Default   2492.000000           run   2492.000000  19.0000  19.0000
368   Default   1548.000000       program   1548.000000  18.0000  18.0000
322   Default   9336.000000         write   9336.000000  17.0000  17.0000
65    Default  10489.000000            do  10489.000000  16.0000  16.0000
201   Default   1235.000000      software   1235.000000  15.0000  15.0000
677   Default   3002.000000       problem   3002.000000  14.0000  14.0000
683   Default   1215.000000          sale   1215.000000  13.0000  13.0000
55    Default   7316.000000       article   7316.000000  12.0000  12.0000
518   Default    882.000000       graphic    882.000000  11.0000  11.0000
303   Default   7035.000000           say   7035.000000  10.0000  10.0000
1862  Default   1010.000000         image   1010.000000   9.0000   9.0000
23    Default   1762.000000          mail   1762.000000   8.0000   8.0000
115   Default   1199.000000         email   1199.000000   7.0000   7.0000
235   Default   1034.000000       version   1034.000000   6.0000   6.0000
353   Default   2191.000000          help   2191.000000   5.0000   5.0000
17    Default   1304.000000          info   1304.000000   4.0000   4.0000
1533  Default    900.000000         color    900.000000   3.0000   3.0000
108   Default   1599.000000           bit   1599.000000   2.0000   2.0000
329   Default   1685.000000       include   1685.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
738    Topic2   1562.604858          file   1859.749512   1.4885  -5.2768
1533   Topic2    787.755066         color    900.526550   1.5288  -5.9617
1543   Topic2    546.881653       monitor    604.665161   1.5622  -6.3266
155    Topic2    678.932678   application    789.458984   1.5118  -6.1103
1862   Topic2    827.764221         image   1010.742920   1.4629  -5.9121
201    Topic2    962.436523      software   1235.004150   1.4133  -5.7614
4141   Topic2    533.836670         video    620.616150   1.5120  -6.3508
235    Topic2    783.776306       version   1034.028198   1.3855  -5.9667
1080   Topic2    580.356445          mode    695.799866   1.4812  -6.2672
683    Topic2    870.865845          sale   1215.140625   1.3295  -5.8614
368    Topic2   1010.799011       program   1548.404541   1.2361  -5.7124
263    Topic2   1245.369629         drive   2091.117920   1.1444  -5.5037
43     Topic2   1259.669678         thank   2435.988281   1.0031  -5.4923
224    Topic2    688.195496        driver    960.635254   1.3291  -6.0968
115    Topic2    768.860535         email   1199.368042   1.2180  -5.9860
317    Topic2   1446.439453           use   4474.748047   0.5333  -5.3540
206    Topic2   1210.598145        system   3252.851074   0.6742  -5.5320
141    Topic2   1009.027405           run   2492.029785   0.7

In [38]:
return pd.concat([default_term_info] + list(topic_dfs))

SyntaxError: 'return' outside function (<ipython-input-38-642e718d7c4b>, line 1)

In [42]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pd.concat([default_term_info] + list(topic_dfs))

NameError: name 'default_term_info' is not defined

In [44]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pd.concat([lda_model, corpus, id2word], sort=True) 
vis

TypeError: cannot concatenate object of type "<class 'gensim.models.ldamodel.LdaModel'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [45]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.336841 -0.001183       1        1  16.240555
13     0.291028 -0.003249       2        1  15.222937
18     0.328831 -0.128123       3        1  14.586671
2      0.091778  0.226472       4        1   6.162434
15     0.059631  0.222477       5        1   5.973762
10    -0.085006 -0.113080       6        1   4.344126
17     0.060749 -0.183168       7        1   4.177632
12    -0.021541 -0.036677       8        1   3.603023
5     -0.022900 -0.021314       9        1   3.365284
16    -0.029244  0.219705      10        1   3.271772
6     -0.062992 -0.016657      11        1   3.222930
9     -0.062884  0.139431      12        1   2.968271
14    -0.088884 -0.062644      13        1   2.740087
3     -0.047752 -0.117436      14        1   2.731137
8     -0.111456  0.003704      15        1   2.291873
19    -0.113684 -0.023385      16        1   2.124455
11    -0.092440 -0.110498      17        1   2.077360
4     -0.134206  0.001651      18        1   1.694628
0     -0.164227 -0.027994      19        1   1.679199
1     -0.131640  0.031969      20        1   1.521866, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
4146   Default  57084.000000            ax  57084.000000  30.0000  30.0000
21     Default  12491.000000          line  12491.000000  29.0000  29.0000
82     Default  16290.000000           not  16290.000000  28.0000  28.0000
30     Default  11142.000000  organization  11142.000000  27.0000  27.0000
3721   Default   3844.000000           max   3844.000000  26.0000  26.0000
65     Default  10452.000000            do  10452.000000  25.0000  25.0000
322    Default   9655.000000         write   9655.000000  24.0000  24.0000
55     Default   7586.000000       article   7586.000000  23.0000  23.0000
244    Default   8046.000000            be   8046.000000  22.0000  22.0000
154    Default  11782.000000         would  11782.000000  21.0000  21.0000
303    Default   6905.000000           say   6905.000000  20.0000  20.0000
226    Default   5803.000000            go   5803.000000  19.0000  19.0000
29     Default   4407.000000     nntp_post   4407.000000  18.0000  18.0000
16     Default   4083.000000          host   4083.000000  17.0000  17.0000
118    Default   6852.000000           get   6852.000000  16.0000  16.0000
46     Default   5027.000000    university   5027.000000  15.0000  15.0000
1911   Default   1863.000000         space   1863.000000  14.0000  14.0000
215    Default   2294.000000        window   2294.000000  13.0000  13.0000
36     Default   3735.000000             s   3735.000000  12.0000  12.0000
18     Default   7291.000000          know   7291.000000  11.0000  11.0000
50     Default   3461.000000          year   3461.000000  10.0000  10.0000
317    Default   4598.000000           use   4598.000000   9.0000   9.0000
2726   Default   1540.000000          game   1540.000000   8.0000   8.0000
23     Default   1949.000000          mail   1949.000000   7.0000   7.0000
681    Default   3005.000000         right   3005.000000   6.0000   6.0000
967    Default   1685.000000     christian   1685.000000   5.0000   5.0000
1362   Default   1746.000000    government   1746.000000   4.0000   4.0000
567    Default   1464.000000          team   1464.000000   3.0000   3.0000
206    Default   3380.000000        system   3380.000000   2.0000   2.0000
60     Default   2012.000000          card   2012.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
11736  Topic20    119.375725          guru    120.327637   4.1773  -5.3260
3548   Topic20    117.628166  countersteer    118.580078   4.1772  -5.3407
10436  Topic20    113.030640       bradley    113.982552   4.1768  -5.3806
280    Topic20    112.221260          lane    113.173172   4.1768  -5

In [1]:

mallet_path = 'F:\\datasets\mallet--2.0.8'  # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

NameError: name 'gensim' is not defined